# Amazon UK Search Results Notebook

## Installations

Importing all necessary modules to run this notebook. Ensure selenium, selectorlib, and fake-useragent have been installed prior to running this notebook.

In [ ]:
import requests
import json
import time
import warnings
import numpy as np
import pandas as pd

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selectorlib import Extractor
from fake_useragent import UserAgent
from selenium.webdriver.support.ui import WebDriverWait

## Loading Pre-Documented Gender Stereotyped Toys

Taking in predoc_stereotyped_items.csv, a CSV file containing 72 rows.

In [ ]:
stereo_toys = pd.read_csv('../predoc_info/predoc_stereotyped_items.csv', delimiter =',')
stereo_toys[:10]

In [ ]:
len(stereo_toys)

## Loading List of Toys Collected from Previous Research

all_items.txt contains a list of strings, where each string represents a toy that will be searched on Amazon UK. This text file contains 166 rows.

In [ ]:
with open('../predoc_info/all_items.txt') as f:
    all_items = f.read().splitlines()

In [ ]:
len(all_items)

## Trial

Creating a short list of 6 toys from all_items. Trial is used so the following functions can be tested on a smaller sample rathen than testing on all 166 toys.

In [ ]:
trial = all_items[160:]
trial

In [ ]:
len(trial)

In [ ]:
generic = ['toys', 'books', 'learning material', 'games', 'sports']

In [ ]:
gender = ['boys', 'girls', 'neutral']

## Scraping Functions

### Unique Identifier Function

This function is used to scrape the ASIN of each toy.

In [ ]:
def asin(driver):
    asin_list = []
    for index in range(1, 10):
        asins = driver.find_elements("xpath", "//div[@data-asin]")
        for asin in asins:
            asin_list.append(asin.get_attribute('data-asin'))
    return asin_list

### Product Title Function

This function is used to scrape the name of each toy.

In [ ]:
def item_info(driver):
    item = []
    elem = driver.find_elements(By.CSS_SELECTOR, 'h2')
    for i in elem:
        item.append(i.text)
    return item

### Product Link Function

This function is used to scrape the associated links of each toy.

In [ ]:
def item_link(driver):
    href = []
    links = driver.find_elements('xpath', "//h2//a[@href]")
    for link in links:
        href.append(link.get_attribute('href'))
    return href

### Search Function

This function calls the above three functions, and runs them on each toy for all three queries (for boys, for girls, for kids).

In [ ]:
def search(item, who):
    if who == 'neutral':
        query = item + ' for ' + 'kids'
    else:
        query = item + ' for ' + who
    driver.get(f'https://www.amazon.co.uk/s?k={query}&ref=nb_sb_noss')
    driver.implicitly_wait(10)
    list_asin = asin(driver)
    item_list = item_info(driver)
    item_page = item_link(driver)
    return (list_asin, item_list), item_page

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
search_result = {}
for i in generic:
    gender_dict = {}
    for g in gender:
        gender_dict[g] = search(i, g)
        driver.implicitly_wait(5)
    search_result[i] = gender_dict
    time.sleep(1.5)
driver.close()

## Database Initialization

Initializing databases to store scraped data.

In [ ]:
columns1 = ['gender', 'query', 'result']
qr = pd.DataFrame(columns = columns1)
columns2 = ['gender', 'query', 'href']
qr_link = pd.DataFrame(columns = columns2)

## Running Queries for Boys, Girls, and Kids (Neutral)

This code is used to scrape all relevant data from the toys included in all_items. As of right now, all_items is used on line 7 in order to run the code through the entire list of toys. Changing all_items with trial on line 7 will faciliate testing as this will run the code on a smaller sample size.

In [ ]:
warnings.filterwarnings('ignore')
driver = webdriver.Chrome(ChromeDriverManager().install())
data1 = []
data2 = []
item = ''
for item in all_items:
    for g in gender:
        result, link = search(item, g)
        values1 = [g, item, result]
        values2 = [g, item, link]
        zipped1 = zip(columns1, values1)
        zipped2 = zip(columns2, values2)
        a_dictionary1 = dict(zipped1)
        a_dictionary2 = dict(zipped2)
        time.sleep(1.5)
        data1.append(a_dictionary1)
        data2.append(a_dictionary2)
driver.close()

Appending ASIN data to previously initialized dataframe.

In [ ]:
qr = qr.append(data1, True)
qr

Database of toys and their associated links.

In [ ]:
qr_link = qr_link.append(data2, True)
qr_link

## Converting Data to CSV File

In [ ]:
az_uk_search_results = pd.DataFrame()

In [ ]:
az_uk_search_results = az_uk_search_results.append(qr, ignore_index =True)
# az_uk_search_results_link = az_uk_search_results_link.append(qr_link, ignore_index =True)

In [ ]:
az_uk_search_results

Export data to CSV file.

In [ ]:
az_uk_search_results.to_csv('az_uk_search_results.csv', index = False)

In [ ]:
df = pd.DataFrame.from_dict(search_result).T.reset_index()
df.rename(columns={'index':'item'}, inplace = True)
df